In [ ]:
import re
import os
import json
from dataclasses import dataclass
from typing import Optional, List, Dict, Tuple
from difflib import SequenceMatcher

try:
    from openai import OpenAI
except ImportError:
    OpenAI = None

# ==========================
# СЛУЖЕБНЫЕ СЛОВАРИ
# ==========================

FORM_SYNONYMS = {
    "таб": "таблетки",
    "табл": "таблетки",
    "таблетка": "таблетки",
    "таблетки": "таблетки",
    "кап": "капсулы",
    "капс": "капсулы",
    "капсула": "капсулы",
    "капсулы": "капсулы",
    "амп": "ампулы",
    "амп.": "ампулы",
    "ампула": "ампулы",
    "ампул": "ампулы",
    "ампулы": "ампулы",
    "amp": "ампулы",
    "ampoule": "ампулы",
    "ampoules": "ампулы",
    "пак": "пакетики",
    "пак.": "пакетики",
    "пакет": "пакетики",
    "пакеты": "пакетики",
    "пакетик": "пакетики",
    "пакетики": "пакетики",
    "саше": "пакетики",
    "саш": "пакетики",
    "sachet": "пакетики",
    "раствор": "раствор",
    "р-р": "раствор",
}

UNIT_SYNONYMS = {
    "mg": "мг",
    "мг": "мг",
    "г": "г",
    "gr": "г",
    "ml": "мл",
    "мл": "мл",
    "мкг": "мкг",
    "mcg": "мкг",
}

STOPWORDS = {
    "для", "по", "в", "и", "от", "при", "против", "без",
    "раствор", "мазь", "спрей", "суспензия", "сироп",
    "таблетки", "капсулы", "пакетики", "пакет", "пак", "порошок",
}

BRAND_FILLER = {
    "от", "для", "при", "против", "без", "n",
    "гриппа", "простуды", "простуда", "кашля", "кашель",
    "внутримышечного", "введения",
}


def load_env_from_dotenv(path: str = ".env") -> None:
    """Простая загрузка переменных из .env без сторонних зависимостей."""
    if not os.path.isfile(path):
        return
    try:
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if not line or line.startswith("#"):
                    continue
                if "=" not in line:
                    continue
                key, value = line.split("=", 1)
                key = key.strip()
                if key and key not in os.environ:
                    os.environ[key] = value.strip().strip("\"").strip("'")
    except OSError:
        pass
NUMERIC_RE = re.compile(r"^\d+(?:[.,]\d+)?$")


@dataclass
class DrugInfo:
    raw: str
    normalized: str
    brand: str
    brand_tokens: List[str]
    dosage_value: Optional[float]
    dosage_unit: Optional[str]
    pack_size: Optional[int]
    form: Optional[str]


def normalize_name(name: str) -> str:
    s = name.lower()

    replacements = {
        "™": " ",
        "®": " ",
        ",": " ",
        ";": " ",
        ":": " ",
        ".": " ",
        "№": " n ",
        "#": " n ",
    }
    for old, new in replacements.items():
        s = s.replace(old, new)

    s = re.sub(r"(№|n)\s*(\d+)", r" n ", s)
    s = re.sub(r"([\d.,]+)\s*(мг|mg|г|gr|мл|ml|мкг|mcg)", r" ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s


def parse_number(token: str) -> Optional[float]:
    if NUMERIC_RE.match(token):
        return float(token.replace(",", "."))
    return None


def normalize_tokens(tokens: List[str]) -> List[str]:
    normalized = []
    for t in tokens:
        if t in UNIT_SYNONYMS:
            t = UNIT_SYNONYMS[t]
        if t in FORM_SYNONYMS:
            t = FORM_SYNONYMS[t]
        normalized.append(t)
    return normalized


def parse_drug(name: str) -> DrugInfo:
    norm = normalize_name(name)
    tokens = normalize_tokens(norm.split())

    dosage_value: Optional[float] = None
    dosage_unit: Optional[str] = None
    pack_size: Optional[int] = None
    form: Optional[str] = None
    used_indices = set()

    for i, t in enumerate(tokens):
        value = parse_number(t)

        if value is not None and i + 1 < len(tokens) and tokens[i + 1] in UNIT_SYNONYMS.values():
            dosage_value = value
            dosage_unit = tokens[i + 1]
            used_indices.update({i, i + 1})

        if t == "n" and i + 1 < len(tokens):
            pack_candidate = parse_number(tokens[i + 1])
            if pack_candidate is not None and pack_candidate.is_integer():
                pack_size = int(pack_candidate)
                used_indices.update({i, i + 1})

        if t in FORM_SYNONYMS.values():
            form = t
            used_indices.add(i)

    if pack_size is None:
        for i in range(len(tokens) - 1, -1, -1):
            if i in used_indices:
                continue
            value = parse_number(tokens[i])
            if value is not None and value.is_integer():
                pack_size = int(value)
                used_indices.add(i)
                break

    brand_tokens: List[str] = []
    for i, t in enumerate(tokens):
        if i in used_indices:
            continue
        if t in STOPWORDS or t in BRAND_FILLER:
            continue
        if parse_number(t) is not None:
            continue
        if t == "n":
            continue
        brand_tokens.append(t)

    brand = " ".join(brand_tokens).strip()

    return DrugInfo(
        raw=name,
        normalized=" ".join(tokens),
        brand=brand,
        brand_tokens=brand_tokens,
        dosage_value=dosage_value,
        dosage_unit=dosage_unit,
        pack_size=pack_size,
        form=form,
    )


def normalize_dosage(value: Optional[float], unit: Optional[str]) -> Tuple[Optional[float], Optional[str]]:
    if value is None or unit is None:
        return None, None
    unit = UNIT_SYNONYMS.get(unit, unit)

    if unit == "г":
        return value * 1000, "мг"
    if unit == "мкг":
        return value / 1000, "мг"
    return value, unit


def compare_brands(tokens1: List[str], tokens2: List[str]) -> Tuple[Optional[bool], Optional[str]]:
    if not tokens1 or not tokens2:
        return None, "Недостаточно данных по бренду"

    set1, set2 = set(tokens1), set(tokens2)
    if set1 == set2:
        return True, None

    if set1 <= set2 or set2 <= set1:
        if len(set1.symmetric_difference(set2)) <= 1:
            return True, "Бренд совпадает, отличается только описание"

    ratio = SequenceMatcher(None, " ".join(sorted(set1)), " ".join(sorted(set2))).ratio()
    if ratio >= 0.82:
        return True, "Бренды похожи по написанию"

    return False, f"Разные бренды: '{' '.join(tokens1)}' vs '{' '.join(tokens2)}'"


def compare_dosage(d1: DrugInfo, d2: DrugInfo) -> Tuple[Optional[bool], Optional[str]]:
    if d1.dosage_value is None or d1.dosage_unit is None or d2.dosage_value is None or d2.dosage_unit is None:
        return None, "Не хватает данных по дозировке"

    v1, base1 = normalize_dosage(d1.dosage_value, d1.dosage_unit)
    v2, base2 = normalize_dosage(d2.dosage_value, d2.dosage_unit)
    if base1 != base2:
        return False, f"Разные единицы дозировки: {d1.dosage_unit} vs {d2.dosage_unit}"

    if v1 == v2:
        return True, None

    return False, f"Разная дозировка: {d1.dosage_value}{d1.dosage_unit or ''} vs {d2.dosage_value}{d2.dosage_unit or ''}"


def compare_form(f1: Optional[str], f2: Optional[str]) -> Tuple[Optional[bool], Optional[str]]:
    if not f1 or not f2:
        return None, "Не хватает данных по форме"
    if f1 == f2:
        return True, None
    return False, f"Разная форма: {f1} vs {f2}"


def compare_pack_sizes(p1: Optional[int], p2: Optional[int]) -> Tuple[Optional[bool], Optional[str]]:
    if p1 is None or p2 is None:
        return None, "Не хватает данных по количеству в упаковке"
    if p1 == p2:
        return True, None
    return False, f"Разный размер упаковки: {p1} vs {p2}"


def rule_compare_drugs(name1: str, name2: str) -> Dict:
    d1 = parse_drug(name1)
    d2 = parse_drug(name2)

    checks = {
        "brand": compare_brands(d1.brand_tokens, d2.brand_tokens),
        "dosage": compare_dosage(d1, d2),
        "form": compare_form(d1.form, d2.form),
        "pack": compare_pack_sizes(d1.pack_size, d2.pack_size),
    }

    reasons: List[str] = []
    has_conflict = False
    for key, (status, reason) in checks.items():
        if status is False:
            has_conflict = True
            if reason:
                reasons.append(reason)

    strong_match = (
        checks["brand"][0] is True
        and checks["dosage"][0] in (True, None)
        and checks["form"][0] in (True, None)
        and checks["pack"][0] in (True, None)
    )

    needs_llm = has_conflict or not strong_match

    return {
        "match": False if has_conflict else strong_match,
        "needs_llm": needs_llm,
        "reasons": reasons,
        "parsed1": d1,
        "parsed2": d2,
        "checks": checks,
    }


def llm_compare_drugs(client: OpenAI, name1: str, name2: str, parsed1: DrugInfo, parsed2: DrugInfo) -> Dict:
    if OpenAI is None:
        raise RuntimeError("Модуль openai не установлен. Установите пакет `openai`, чтобы использовать LLM.")

    prompt = f"""
Ты помогаешь сопоставлять медикаменты из разных баз.
Используй приведённые разобранные данные и сами исходные названия.

Название 1: "{parsed1.raw}"
Нормализовано 1: "{parsed1.normalized}"
brand: "{parsed1.brand}", dose: "{parsed1.dosage_value} {parsed1.dosage_unit}", pack: "{parsed1.pack_size}", form: "{parsed1.form}"

Название 2: "{parsed2.raw}"
Нормализовано 2: "{parsed2.normalized}"
brand: "{parsed2.brand}", dose: "{parsed2.dosage_value} {parsed2.dosage_unit}", pack: "{parsed2.pack_size}", form: "{parsed2.form}"

Ответь строго JSON (без комментариев и текста вокруг):
{{
  "match": true/false,
  "reason": "краткое объяснение на русском"
}}

Логика:
- Сначала опирайся на бренд/торговое название, дозировку, форму и упаковку из структурированных полей.
- Учитывай эквивалентность дозировок (пример: 0.1 г = 100 мг).
- Если нет явных противоречий и бренды/дозировки совпадают - match = true.
- Если формы или дозировки отличаются - match = false.
- Если информация неполная, используй исходные названия, но не придумывай несуществующие данные.
- Никаких пояснений вне JSON.
"""


    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
    )

    content = response.choices[0].message.content
    try:
        data = json.loads(content)
    except json.JSONDecodeError:
        data = {"match": False, "reason": "Не удалось разобрать ответ модели", "raw": content}

    return data


def smart_compare_drugs(name1: str, name2: str) -> Dict:
    rule_result = rule_compare_drugs(name1, name2)

    print("=== Разбор 1 ===")
    print(rule_result["parsed1"])
    print("=== Разбор 2 ===")
    print(rule_result["parsed2"])
    print("===")

    if not rule_result["needs_llm"]:
        decision = "совпадают" if rule_result["match"] else "НЕ совпадают"
        print(f"✅ Правила решили, что препараты {decision}.")
        if rule_result["reasons"]:
            print("Причины/заметки:", rule_result["reasons"])
        return {
            "match": rule_result["match"],
            "source": "rules",
            "rule_reasons": rule_result["reasons"],
        }

    print("⚠ Правила не уверены (не хватает данных), зовём LLM.")
    if rule_result["reasons"]:
        print("Причины (по правилам):")
        for r in rule_result["reasons"]:
            print(" -", r)

    load_env_from_dotenv()
    api_key = os.getenv("OPENAI_API_KEY")
    if not api_key:
        print("⚠ LLM не вызван: нет OPENAI_API_KEY")
        return {
            "match": rule_result["match"],
            "source": "rules",
            "rule_reasons": rule_result["reasons"],
            "llm_reason": "LLM недоступен: нет OPENAI_API_KEY",
        }
    if OpenAI is None:
        print("⚠ LLM не вызван: модуль openai не установлен")
        return {
            "match": rule_result["match"],
            "source": "rules",
            "rule_reasons": rule_result["reasons"],
            "llm_reason": "LLM недоступен: нет модуля openai",
        }

    client = OpenAI(api_key=api_key)
    llm_result = llm_compare_drugs(
        client,
        name1,
        name2,
        rule_result["parsed1"],
        rule_result["parsed2"],
    )

    print("\n=== Ответ LLM ===")
    print(llm_result)

    return {
        "match": bool(llm_result.get("match", False)),
        "source": "llm",
        "rule_reasons": rule_result["reasons"],
        "llm_reason": llm_result.get("reason"),
    }


if __name__ == "__main__":
    name_a = "Беневрон Б раствор для внутримышечного введения 3 мл №5"
    name_b = "Беневрон Б 3мл амп №5"

    result = smart_compare_drugs(name_a, name_b)
    print("\n=== ИТОГО ===")
    print("Совпадают?", result["match"])
    print("Источник решения:", result["source"])
